# Book Recommendation System - Part IV
## by: Amir Momen-Roknabadi
### The Data Incubator - June 2021 Part Time Cohort

## Testing the efficiency of the trained system:
Prveiously, I was only looking at the users that were in my training DB. Now, I will create a new test set. These will be from users that my system has not seen before. My aim is now to ensure that I can provide a recommendation for a user that I have not seen before. First, I tried using a combination of PCA/ KNN approach to find the most similar users from my training set to the new user. But this approach did not perform well. In the next step, I used transfer learning to train the model on the new users. 

I will divide up the rest of the data set to a new training and a test set  (df_train2, and df_test2):

In [1]:
import pandas as pd
import numpy as np
import math
import glob
from sklearn.preprocessing import LabelEncoder
from plotnine import *
import random
from Model_twostate import *
from torchsummary import summary
from IPython.display import Image
from random import seed
from random import random
from random import sample
%config Completer.use_jedi = False 
df_train = pd.read_csv('/mnt/data1/amir/Amazon/Amazontrain-20percent.csv', index_col=0, header=0)
df_valid = pd.read_csv('/mnt/data1/amir/Amazon/Amazonvalid-20percent.csv', index_col=0, header=0)
df_test = pd.read_csv('/mnt/data1/amir/Amazon/Amazontest-20percent.csv', index_col=0, header=0)
rating=pd.read_csv('/mnt/data1/amir/Amazon/Rating.csv', index_col=0, header=0)
books_size=5860902
users_size=30725236#

/home/amir/.local/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


I will combine the DB used to train the network.

In [17]:
pretrainedDS = pd.concat([df_train, df_valid, df_test])
pretrainedDS.head()

,ASIN,ID,Rating,Rating_id,NumericRating,Book_id,NumericID
3660013,0140447938,AZZY4W8E5AX2K,1.0,1455667200,1,97962,15362458
3851620,0141331976,AZZY4W8E5AX2K,1.0,1455667200,1,100795,15362458
3883378,0141354224,AZZY4W8E5AX2K,1.0,1455667200,1,100974,15362458
5135663,0297870920,AZZY4W8E5AX2K,1.0,1433808000,1,169084,15362458
11107913,044651862X,AZZY4W8E5AX2K,1.0,1427760000,1,346071,15362458


Now, I will filter my rating DB to make sure I am only looking at the same books

In [18]:
filteredwithBookIDinpretrained= morethan10rating[rating.Book_id.isin(pretrainedDS['Book_id'])]
filteredwithBookIDinpretrained

/home/amir/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,ASIN,ID,Rating,Rating_id,NumericRating,Book_id,NumericID
138,0002005263,A2VDN6OJOIY74W,0.0,1052697600,0,273,7600077
139,0002005263,A1POJBC0L210UT,0.0,1052697600,0,273,2898942
140,0002005263,A2X9XKFB18UQCU,0.0,1052524800,0,273,7814129
143,0002005263,A25SQVXZL0OICV,1.0,1500768000,1,273,4717267
145,0002005263,A9KF6TVHK49XH,1.0,1497225600,1,273,12385257
...,...,...,...,...,...,...,...
51061544,B01HFWA3OS,AKRUM53ZRQG1D,0.0,1509062400,0,2930196,13647676
51061611,B01HGK2QRQ,A1IMELIVCP8MCL,0.0,1504569600,0,2930236,2102536
51061612,B01HGK2QRQ,A2FEPB4A0CWUIP,1.0,1504396800,1,2930236,5800935
51061616,B01HGK2QRQ,A1QPMX67RRD77V,1.0,1481500800,1,2930236,3015424


Now that the ratings are filtered, I will look create a filter for more than 10 books per user.

In [19]:
summary=filteredwithBookIDinpretrained[['ID', 'NumericRating']].groupby(['ID']).agg(['count'])
summary.columns=['count']
summary

,count
ID,
A00037304EKN1SJNQV5A,11
A000392684P4JNLQRBBW,11
A0015332H21AK8WZ0ZCS,12
A0042814FUEOXHEB05SQ,13
A0044294CPA1QY04FSGM,14
...,...
AZZXSBASANYPU,14
AZZY4E6QPTBVP,17
AZZY4W8E5AX2K,16


In [20]:
filteredMorethan10= morethan10rating[~morethan10rating.ID.isin(pretrainedDS['ID'])]
filteredMorethan10.head()

,ASIN,ID,Rating,Rating_id,NumericRating,Book_id,NumericID
140,0002005263,A2X9XKFB18UQCU,0.0,1052524800,0,273,7814129
145,0002005263,A9KF6TVHK49XH,1.0,1497225600,1,273,12385257
149,0002005263,A353L9MM29O3UV,1.0,1493683200,1,273,8696584
155,0002005263,A3LAQSHW9IVKUO,1.0,1478390400,1,273,10521558
156,0002005263,A21OVLZW4BFF6R,1.0,1474156800,1,273,4253890


In [21]:
morethan10rating.shape,lessthan10rating.shape,filteredMorethan10.shape, rating.shape

((8230206, 7), (34319713, 7), (4940894, 7), (51062215, 7))

I will select 10% of the total remaining ratings as my new test/ training set. Out of which, 10% will be the test set. 

In [22]:
g = filteredMorethan10.groupby('ID', as_index=False)

In [23]:
df_test2 = g.head(2)
df_test2

,ASIN,ID,Rating,Rating_id,NumericRating,Book_id,NumericID
140,0002005263,A2X9XKFB18UQCU,0.0,1052524800,0,273,7814129
145,0002005263,A9KF6TVHK49XH,1.0,1497225600,1,273,12385257
149,0002005263,A353L9MM29O3UV,1.0,1493683200,1,273,8696584
155,0002005263,A3LAQSHW9IVKUO,1.0,1478390400,1,273,10521558
156,0002005263,A21OVLZW4BFF6R,1.0,1474156800,1,273,4253890
...,...,...,...,...,...,...,...
45565890,1535301163,A101RUD0LFCSN,0.0,1526169600,0,1804642,10106
47278439,1681854473,A1VCXRPOYQ3LM9,0.0,1508803200,0,2186041,3539853
47280686,1681857537,A1VCXRPOYQ3LM9,0.0,1508803200,0,2186085,3539853
48077469,1936185210,AZ2VIM7ZGIB,1.0,1519344000,1,2402348,15259305


In [24]:
df_train2 = filteredMorethan10[~filteredMorethan10.index.isin(df_test2.index)]
df_train2

,ASIN,ID,Rating,Rating_id,NumericRating,Book_id,NumericID
11610,0001844423,AT1UXZSMP6D6V,1.0,1306972800,1,160,14580041
15024,0001951076,A24GN9L1XL37VU,1.0,1498003200,1,213,4566686
15082,0001951076,A2IKGNPO6QS8W0,1.0,1489017600,1,213,6157811
16987,0001951076,AQU0ZW5D9V1V0,1.0,1071360000,1,213,14329844
17082,0001713256,A2ZGZXY0EIZS8S,1.0,1494633600,1,121,8061383
...,...,...,...,...,...,...,...
51061540,B01HFWA3OS,A2D38JW44EDSQ2,1.0,1514073600,1,2930196,5539802
51061543,B01HFWA3OS,A30Y76WENLXFXS,0.0,1510963200,0,2930196,8228227
51061544,B01HFWA3OS,AKRUM53ZRQG1D,0.0,1509062400,0,2930196,13647676
51061611,B01HGK2QRQ,A1IMELIVCP8MCL,0.0,1504569600,0,2930236,2102536


In [25]:
df_train2.to_csv('/mnt/data1/amir/Amazon/Amazontrain-forfinaltesting-10percent.csv')
df_test2.to_csv('/mnt/data1/amir/Amazon/Amazontest-forfinaltesting-10percent.csv')


## Testing using PCA/ KNN approach

Now, I need to create a pipeline to find the users from the training set of the neural network that are similar to users of this train/test set that the neural network has not seen before:

In [2]:
import pandas as pd
import numpy as np
import math
import glob
from sklearn.preprocessing import LabelEncoder
from plotnine import *
import random
from Model_twostate import *
from torchsummary import summary
from IPython.display import Image
from random import seed
from random import random
from random import sample
%config Completer.use_jedi = False 
df_train = pd.read_csv('/mnt/data1/amir/Amazon/Amazontrain-20percent.csv', index_col=0, header=0)
df_valid = pd.read_csv('/mnt/data1/amir/Amazon/Amazonvalid-20percent.csv', index_col=0, header=0)
df_test = pd.read_csv('/mnt/data1/amir/Amazon/Amazontest-20percent.csv', index_col=0, header=0)
rating=pd.read_csv('/mnt/data1/amir/Amazon/Rating.csv', index_col=0, header=0)
books_size=5860902
users_size=30725236
df_train2 = pd.read_csv('/mnt/data1/amir/Amazon/Amazontrain-forfinaltesting-10percent.csv', index_col=0, header=0)
df_test2 = pd.read_csv('/mnt/data1/amir/Amazon/Amazontest-forfinaltesting-10percent.csv', index_col=0, header=0)


In [3]:
summarytest=df_test2[['NumericRating', 'ID']].groupby(['NumericRating']).agg(['count'])
summarytest.iloc[1,:]/(summarytest.iloc[1,:]+summarytest.iloc[0,:])

ID  count    0.646911
dtype: float64

 I need to create a base dictionary to ensure I have all the possible books

In [4]:
BookIDs=rating['Book_id'].unique()
BookIDs.shape

(2930451,)

In [5]:
baseDict={el:0 for el in BookIDs}

Now that I have the base dict I need to create a function to convert the dataframe to dictionary

In [14]:
def pdtodict(arr):
    """this function will convert an array of books and thier rating to a dictionary. This is to be used with a dictvectorizer"""  
    out={x:y*1000+100 for x,y in zip(arr['Book_id'], arr['NumericRating'])} ### I multiplied them by 1000 and added 100 to distinguish them from the books that the user has not seen
    return out

 Now, I will convert the df_train to a list of dictionaries

In [8]:
listofDicts=[pdtodict(df_train.loc[df_train.NumericID==ID]) for ID in NumericIDs]

In [10]:
listofDicts.append(baseDict)

In [11]:
import pickle
with open("listofDicts.pkl", "wb") as fp:
    pickle.dump(listofDicts, fp)

In [13]:
listofDicts_train2=[pdtodict(df_train2.loc[df_train2.NumericID==ID]) for ID in NumericIDs2]

In [14]:
with open("listofDicts2.pkl", "wb") as fp:
    pickle.dump(listofDicts_train2, fp)

Now, I need to use dictvectorizer to prepare the data for KNN:

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np
import math
import glob
from sklearn.preprocessing import LabelEncoder
from plotnine import *
import random
from Model_twostate import *
from torchsummary import summary
from IPython.display import Image
from random import seed
from random import random
from random import sample
import pickle

%config Completer.use_jedi = False 
df_train = pd.read_csv('/mnt/data1/amir/Amazon/Amazontrain-20percent.csv', index_col=0, header=0)
df_valid = pd.read_csv('/mnt/data1/amir/Amazon/Amazonvalid-20percent.csv', index_col=0, header=0)
df_test = pd.read_csv('/mnt/data1/amir/Amazon/Amazontest-20percent.csv', index_col=0, header=0)
rating=pd.read_csv('/mnt/data1/amir/Amazon/Rating.csv', index_col=0, header=0)
books_size=5860902
users_size=30725236
df_train2 = pd.read_csv('/mnt/data1/amir/Amazon/Amazontrain-forfinaltesting-10percent.csv', index_col=0, header=0)
df_test2 = pd.read_csv('/mnt/data1/amir/Amazon/Amazontest-forfinaltesting-10percent.csv', index_col=0, header=0)
with open("listofDicts.pkl", "rb") as fp:   # Unpickling
    listofDicts = pickle.load(fp)
with open("listofDicts2.pkl", "rb") as fp:   # Unpickling
    listofDicts_train2 = pickle.load(fp) 

In [3]:
NumericIDs2=df_train2['NumericID'].unique()

In [5]:
NumericIDs=df_train['NumericID'].unique()
NumericIDs.shape

(171192,)

In [6]:
y=NumericIDs.ravel()
y=np.append(y,[9999999999999999])

In [7]:
VectorDict=DictVectorizer()
spca = TruncatedSVD(n_components=100)
neigh = KNeighborsClassifier(n_neighbors=1, algorithm = 'brute', n_jobs=5)

In [8]:
pipe = Pipeline(steps=[('pddict2vect', VectorDict), ('pca',spca), ('m',neigh)])

In [9]:
len(listofDicts)

171193

In [10]:
pipe.fit(listofDicts, y)

Pipeline(steps=[('pddict2vect', DictVectorizer()),
                ('pca', TruncatedSVD(n_components=100)),
                ('m',
                 KNeighborsClassifier(algorithm='brute', n_jobs=5,
                                      n_neighbors=1))])

In [11]:
spca.explained_variance_ratio_

array([0.00345312, 0.00162862, 0.00168065, 0.00119456, 0.00120854,
       0.00116311, 0.00111871, 0.00108952, 0.0010539 , 0.00103887,
       0.00099236, 0.00097001, 0.00096016, 0.00094026, 0.00090429,
       0.00089426, 0.00086914, 0.00085332, 0.00079953, 0.00077649,
       0.00075535, 0.00074367, 0.00071447, 0.000707  , 0.00069284,
       0.00065974, 0.00062396, 0.00059496, 0.000594  , 0.00058523,
       0.00057042, 0.0005601 , 0.00055157, 0.00054295, 0.00053621,
       0.00053125, 0.00052882, 0.00052547, 0.00051984, 0.00051614,
       0.00051466, 0.00050976, 0.00050861, 0.00049865, 0.00049305,
       0.00049353, 0.00049097, 0.00048315, 0.00047577, 0.00047555,
       0.00046815, 0.0004659 , 0.00046334, 0.00045813, 0.00045248,
       0.0004487 , 0.00044709, 0.00044433, 0.00044363, 0.00043799,
       0.00043904, 0.00043662, 0.00042943, 0.00042659, 0.00042367,
       0.00041577, 0.00041207, 0.00040662, 0.0004032 , 0.00039534,
       0.00039294, 0.00038973, 0.0003829 , 0.00038065, 0.00037

In [12]:
temp=df_train2.loc[df_train2.NumericID==7814129,['Book_id', 'NumericRating']]

In [15]:
pdtodict(temp)

{273: 100,
 4239: 1100,
 4530: 1100,
 17732: 1100,
 18401: 1100,
 22504: 100,
 99286: 100,
 183813: 1100,
 445429: 1100,
 445447: 1100,
 445233: 100,
 450739: 100,
 451070: 1100,
 452155: 1100}

In [16]:
num=int(pipe.predict(pdtodict(temp)))
print(num)

11624163


In [17]:
temp2=df_train.loc[df_train.NumericID==num,['Book_id', 'NumericRating']]
pdtodict(temp2)

{363890: 1100,
 917: 1100,
 4861: 1100,
 104326: 1100,
 244866: 1100,
 4239: 1100,
 1949048: 1100}

In [18]:
predicted = pipe.predict(listofDicts_train2)

In [19]:
predicted

array([11624163, 11897865,  9812785, ...,  5166208,  6033130, 11315834])

In [20]:
UserIDConv = pd.DataFrame(data={'NumericID': NumericIDs2, 'conv':predicted})
UserIDConv

,NumericID,conv
0,7814129,11624163
1,12385257,11897865
2,8696584,9812785
3,10521558,4436835
4,4253890,1683127
...,...,...
256783,4713541,6516832
256784,6313771,8135453
256785,4360317,5166208
256786,3539853,6033130


In [21]:
df_test2 = df_test2.merge(UserIDConv, how='inner')
df_test2

,ASIN,ID,Rating,Rating_id,NumericRating,Book_id,NumericID,conv
0,0002005263,A2X9XKFB18UQCU,0.0,1052524800,0,273,7814129,11624163
1,0007196997,A2X9XKFB18UQCU,1.0,1134000000,1,4239,7814129,11624163
2,0002005263,A9KF6TVHK49XH,1.0,1497225600,1,273,12385257,11897865
3,0001844423,A9KF6TVHK49XH,1.0,1518739200,1,160,12385257,11897865
4,0002005263,A353L9MM29O3UV,1.0,1493683200,1,273,8696584,9812785
...,...,...,...,...,...,...,...,...
513571,1481281429,A22MX91QCBU2MC,1.0,1526083200,1,1593092,4360317,5166208
513572,1681854473,A1VCXRPOYQ3LM9,0.0,1508803200,0,2186041,3539853,6033130
513573,1681857537,A1VCXRPOYQ3LM9,0.0,1508803200,0,2186085,3539853,6033130
513574,1936185210,AZ2VIM7ZGIB,1.0,1519344000,1,2402348,15259305,11315834


In [23]:
books_size=5860902
users_size=30725236
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")
device = torch.device("cpu")
model = BookRecommender(books_size=books_size, users_size=users_size, pdropbooks=0.7, pdropusers = 0.1, pdropfc=0.5, books_dim=12,  users_dim=12, fchidden1=16, fchidden2=32, lr =0.00217058)
print(model)
model.compile(device="cpu")

BookRecommender(
  (booklayer): Sequential(
    (0): Embedding(5860902, 12)
    (1): Dropout(p=0.7, inplace=False)
  )
  (userlayer): Sequential(
    (0): Embedding(30725236, 12)
    (1): Dropout(p=0.1, inplace=False)
  )
  (conv_block_1): Sequential(
    (0): Conv1d(8, 1, kernel_size=(10,), stride=(1,), padding=(10,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool1d(kernel_size=10, stride=5, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.2, inplace=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=24, out_features=16, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=16, out_features=32, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=32, out_features=1, bias=True)
  )
)


In [24]:
test_data_loader = DataLoader(torch.utils.data.TensorDataset(torch.from_numpy(df_test2["Book_id"].values), torch.from_numpy(df_test2["conv"].values), torch.from_numpy(df_test2["NumericRating"].values)), batch_size=1028, shuffle=False, drop_last=False)

In [26]:
model.load_model('models/best_ret_checkpoint.pt')

In [27]:
model.eval()
target_list = []
pred_list = []
with torch.no_grad():
    for batch_idx, batch in enumerate(test_data_loader):
        books,users, ratings = batch
        books = books.to(device)
        users = users.to(device)
        out = model(books, users)
        pred_list.append(out.to('cpu').detach().numpy())
        target_list.append(ratings.to('cpu').detach().numpy())
    targets = np.concatenate(target_list)
    preds = np.concatenate(pred_list)
    valid_acc = accuracy2(preds, targets)

In [28]:
valid_acc

0.6453202641868

In [29]:
summary_test_ratings=df_test2[['ID', 'NumericRating']].groupby(['NumericRating']).agg(['count'])
summary_test_ratings.columns=["counts"]
summary_test_ratings.iloc[1]/(summary_test_ratings.iloc[1]+summary_test_ratings.iloc[0])

counts    0.646911
dtype: float64

The prediction accuracy of my KNN model was 64.53% but the 64.69% of the test set had a value of 1. So, this means that my model is performing worse than a naive approach of recommending all the books. I will try to change the number of components:

In [34]:
valid_acc = np.zeros(101)
for ncomp in range(3,16):
    VectorDict=DictVectorizer()
    spca = TruncatedSVD(n_components=ncomp)
    neigh = KNeighborsClassifier(n_neighbors=1, algorithm = 'brute', n_jobs=5)
    pipe = Pipeline(steps=[('pddict2vect', VectorDict), ('pca',spca), ('m',neigh)])
    pipe.fit(listofDicts, y)
    predicted = pipe.predict(listofDicts_train2)
    UserIDConv = pd.DataFrame(data={'NumericID': NumericIDs2, 'conv':predicted})
    testset = df_test2.merge(UserIDConv, how='inner')
    device = torch.device("cpu")
    model = BookRecommender(books_size=books_size, users_size=users_size, pdropbooks=0.7, pdropusers = 0.1, pdropfc=0.5, books_dim=12,  users_dim=12, fchidden1=16, fchidden2=32, lr =0.00217058)
    model.compile(device=device)
    test_data_loader = DataLoader(torch.utils.data.TensorDataset(torch.from_numpy(testset["Book_id"].values), torch.from_numpy(testset["conv"].values), torch.from_numpy(testset["NumericRating"].values)), batch_size=1028, shuffle=False, drop_last=False)
    model.load_model('models/best_ret_checkpoint.pt')
    model.eval()
    target_list = []
    pred_list = []
    with torch.no_grad():
        for batch_idx, batch in enumerate(test_data_loader):
            books,users, ratings = batch
            books = books.to(device)
            users = users.to(device)
            out = model(books, users)
            pred_list.append(out.to('cpu').detach().numpy())
            target_list.append(ratings.to('cpu').detach().numpy())
        targets = np.concatenate(target_list)
        preds = np.concatenate(pred_list)
        valid_acc[ncomp] = accuracy2(preds, targets)
        print("The accuracy for " + str(ncomp) +" components in Truncated SVD is:" + str(valid_acc[ncomp]))

The accuracy for 3 components in Truncated SVD is:0.6084961135255541
The accuracy for 4 components in Truncated SVD is:0.6092165521753353
The accuracy for 5 components in Truncated SVD is:0.6133230524790878
The accuracy for 6 components in Truncated SVD is:0.6188275931897129
The accuracy for 7 components in Truncated SVD is:0.6224901475146813
The accuracy for 8 components in Truncated SVD is:0.6273579762294188
The accuracy for 9 components in Truncated SVD is:0.6280608906958269
The accuracy for 10 components in Truncated SVD is:0.6298853528981105
The accuracy for 11 components in Truncated SVD is:0.6314586351387137
The accuracy for 12 components in Truncated SVD is:0.6333220399707151
The accuracy for 13 components in Truncated SVD is:0.634102839696559
The accuracy for 14 components in Truncated SVD is:0.6337114662678941
The accuracy for 15 components in Truncated SVD is:0.6347609701387915


In [35]:
for ncomp in range(16,31):
    VectorDict=DictVectorizer()
    spca = TruncatedSVD(n_components=ncomp)
    neigh = KNeighborsClassifier(n_neighbors=1, algorithm = 'brute', n_jobs=5)
    pipe = Pipeline(steps=[('pddict2vect', VectorDict), ('pca',spca), ('m',neigh)])
    pipe.fit(listofDicts, y)
    predicted = pipe.predict(listofDicts_train2)
    UserIDConv = pd.DataFrame(data={'NumericID': NumericIDs2, 'conv':predicted})
    testset = df_test2.merge(UserIDConv, how='inner')
    device = torch.device("cpu")
    model = BookRecommender(books_size=books_size, users_size=users_size, pdropbooks=0.7, pdropusers = 0.1, pdropfc=0.5, books_dim=12,  users_dim=12, fchidden1=16, fchidden2=32, lr =0.00217058)
    model.compile(device=device)
    test_data_loader = DataLoader(torch.utils.data.TensorDataset(torch.from_numpy(testset["Book_id"].values), torch.from_numpy(testset["conv"].values), torch.from_numpy(testset["NumericRating"].values)), batch_size=1028, shuffle=False, drop_last=False)
    model.load_model('models/best_ret_checkpoint.pt')
    model.eval()
    target_list = []
    pred_list = []
    with torch.no_grad():
        for batch_idx, batch in enumerate(test_data_loader):
            books,users, ratings = batch
            books = books.to(device)
            users = users.to(device)
            out = model(books, users)
            pred_list.append(out.to('cpu').detach().numpy())
            target_list.append(ratings.to('cpu').detach().numpy())
        targets = np.concatenate(target_list)
        preds = np.concatenate(pred_list)
        valid_acc[ncomp] = accuracy2(preds, targets)
        print("The accuracy for " + str(ncomp) +" components in Truncated SVD is:" + str(valid_acc[ncomp]))

The accuracy for 16 components in Truncated SVD is:0.6357248002243095
The accuracy for 17 components in Truncated SVD is:0.6355826596258393
The accuracy for 18 components in Truncated SVD is:0.6358124211411749
The accuracy for 19 components in Truncated SVD is:0.6370196426624297
The accuracy for 20 components in Truncated SVD is:0.6358552580338646
The accuracy for 21 components in Truncated SVD is:0.6379562129071452
The accuracy for 22 components in Truncated SVD is:0.638127560477904
The accuracy for 23 components in Truncated SVD is:0.6378802747791953
The accuracy for 24 components in Truncated SVD is:0.6394730283346574
The accuracy for 25 components in Truncated SVD is:0.6399500755487016
The accuracy for 26 components in Truncated SVD is:0.6406159945168778
The accuracy for 27 components in Truncated SVD is:0.6412059753571039
The accuracy for 28 components in Truncated SVD is:0.6412059753571039
The accuracy for 29 components in Truncated SVD is:0.6414396311354114
The accuracy for 30 c

In [36]:
for ncomp in range(31,50):
    VectorDict=DictVectorizer()
    spca = TruncatedSVD(n_components=ncomp)
    neigh = KNeighborsClassifier(n_neighbors=1, algorithm = 'brute', n_jobs=5)
    pipe = Pipeline(steps=[('pddict2vect', VectorDict), ('pca',spca), ('m',neigh)])
    pipe.fit(listofDicts, y)
    predicted = pipe.predict(listofDicts_train2)
    UserIDConv = pd.DataFrame(data={'NumericID': NumericIDs2, 'conv':predicted})
    testset = df_test2.merge(UserIDConv, how='inner')
    device = torch.device("cpu")
    model = BookRecommender(books_size=books_size, users_size=users_size, pdropbooks=0.7, pdropusers = 0.1, pdropfc=0.5, books_dim=12,  users_dim=12, fchidden1=16, fchidden2=32, lr =0.00217058)
    model.compile(device=device)
    test_data_loader = DataLoader(torch.utils.data.TensorDataset(torch.from_numpy(testset["Book_id"].values), torch.from_numpy(testset["conv"].values), torch.from_numpy(testset["NumericRating"].values)), batch_size=1028, shuffle=False, drop_last=False)
    model.load_model('models/best_ret_checkpoint.pt')
    model.eval()
    target_list = []
    pred_list = []
    with torch.no_grad():
        for batch_idx, batch in enumerate(test_data_loader):
            books,users, ratings = batch
            books = books.to(device)
            users = users.to(device)
            out = model(books, users)
            pred_list.append(out.to('cpu').detach().numpy())
            target_list.append(ratings.to('cpu').detach().numpy())
        targets = np.concatenate(target_list)
        preds = np.concatenate(pred_list)
        valid_acc[ncomp] = accuracy2(preds, targets)
        print("The accuracy for " + str(ncomp) +" components in Truncated SVD is:" + str(valid_acc[ncomp]))

The accuracy for 31 components in Truncated SVD is:0.6417336479897815
The accuracy for 32 components in Truncated SVD is:0.6428337772793121
The accuracy for 33 components in Truncated SVD is:0.6447244419521161
The accuracy for 34 components in Truncated SVD is:0.6445511472498715
The accuracy for 35 components in Truncated SVD is:0.6434276523825101
The accuracy for 36 components in Truncated SVD is:0.6443428041808807
The accuracy for 37 components in Truncated SVD is:0.6439475364892441
The accuracy for 38 components in Truncated SVD is:0.6435873171643535
The accuracy for 39 components in Truncated SVD is:0.6448120628689814
The accuracy for 40 components in Truncated SVD is:0.6445511472498715
The accuracy for 41 components in Truncated SVD is:0.6442766017103603
The accuracy for 42 components in Truncated SVD is:0.6453806252628628
The accuracy for 43 components in Truncated SVD is:0.644726389083602
The accuracy for 44 components in Truncated SVD is:0.6458459896879917
The accuracy for 45 c

## Transfer learning:

Using ```TrunctaedSVD``` combined with KNN to find similar users does not seem to perform well. So, I will look at transfer learning now. To do this, I saved the previous checkpoint as ```best_ret_checkpoint_fullytrained.pt```. I will also create a new validation set from the training set.

In [26]:
df_train2

,ASIN,ID,Rating,Rating_id,NumericRating,Book_id,NumericID
11610,0001844423,AT1UXZSMP6D6V,1.0,1306972800,1,160,14580041
15024,0001951076,A24GN9L1XL37VU,1.0,1498003200,1,213,4566686
15082,0001951076,A2IKGNPO6QS8W0,1.0,1489017600,1,213,6157811
16987,0001951076,AQU0ZW5D9V1V0,1.0,1071360000,1,213,14329844
17082,0001713256,A2ZGZXY0EIZS8S,1.0,1494633600,1,121,8061383
...,...,...,...,...,...,...,...
51061540,B01HFWA3OS,A2D38JW44EDSQ2,1.0,1514073600,1,2930196,5539802
51061543,B01HFWA3OS,A30Y76WENLXFXS,0.0,1510963200,0,2930196,8228227
51061544,B01HFWA3OS,AKRUM53ZRQG1D,0.0,1509062400,0,2930196,13647676
51061611,B01HGK2QRQ,A1IMELIVCP8MCL,0.0,1504569600,0,2930236,2102536


In [27]:
g = df_train2.groupby('ID', as_index=False)

In [28]:
df_valid2 = g.head(1)
df_valid2

,ASIN,ID,Rating,Rating_id,NumericRating,Book_id,NumericID
11610,0001844423,AT1UXZSMP6D6V,1.0,1306972800,1,160,14580041
15024,0001951076,A24GN9L1XL37VU,1.0,1498003200,1,213,4566686
15082,0001951076,A2IKGNPO6QS8W0,1.0,1489017600,1,213,6157811
16987,0001951076,AQU0ZW5D9V1V0,1.0,1071360000,1,213,14329844
17082,0001713256,A2ZGZXY0EIZS8S,1.0,1494633600,1,121,8061383
...,...,...,...,...,...,...,...
45351456,1517245885,A3LI8R90KHS81X,1.0,1514246400,1,1746156,10545188
45378443,1518791654,A1D37DP87WHEI3,1.0,1524614400,1,1753184,1478903
45614691,1539594394,A101RUD0LFCSN,0.0,1526169600,0,1819584,10106
47290740,1682604586,A1VCXRPOYQ3LM9,0.0,1508803200,0,2188856,3539853


In [29]:
df_train3 = df_train2[~df_train2.index.isin(df_valid2.index)]
df_train3

,ASIN,ID,Rating,Rating_id,NumericRating,Book_id,NumericID
26051,0002173611,AQU0ZW5D9V1V0,0.0,1067644800,0,537,14329844
38722,0001050230,A235IF7V4XS5HB,0.0,1493856000,0,39,4418625
41794,0002247437,AT1UXZSMP6D6V,0.0,1328054400,0,839,14580041
46698,000172066X,A24GN9L1XL37VU,1.0,1498003200,1,137,4566686
47378,000172066X,A2ZGZXY0EIZS8S,1.0,1458000000,1,137,8061383
...,...,...,...,...,...,...,...
51061540,B01HFWA3OS,A2D38JW44EDSQ2,1.0,1514073600,1,2930196,5539802
51061543,B01HFWA3OS,A30Y76WENLXFXS,0.0,1510963200,0,2930196,8228227
51061544,B01HFWA3OS,AKRUM53ZRQG1D,0.0,1509062400,0,2930196,13647676
51061611,B01HGK2QRQ,A1IMELIVCP8MCL,0.0,1504569600,0,2930236,2102536


In [30]:
model = BookRecommender(books_size=books_size, users_size=users_size, pdropbooks=0.7, pdropusers = 0.1, pdropfc=0.5, books_dim=12,  users_dim=12, fchidden1=16, fchidden2=32, lr =0.00217058)
books_size=5860902
users_size=30725236
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")
model.load_state_dict(torch.load('models/best_ret_checkpoint_fullytrained.pt'))


<All keys matched successfully>

In [31]:
train_data_loader = DataLoader(torch.utils.data.TensorDataset(torch.from_numpy(df_train3["Book_id"].values), torch.from_numpy(df_train3["NumericID"].values), torch.from_numpy(df_train3["NumericRating"].values)), batch_size=128, shuffle=True, drop_last=True)
valid_data_loader = DataLoader(torch.utils.data.TensorDataset(torch.from_numpy(df_valid2["Book_id"].values), torch.from_numpy(df_valid2["NumericID"].values), torch.from_numpy(df_valid2["NumericRating"].values)), batch_size=128, shuffle=False, drop_last=False)


I removed the output for the following cell to aid with the readability

In [ ]:
model.compile(device=device)
logfile = open('models/Amazonhistory3.log', 'at', buffering=1)
hist = model.train_model(train_data_loader,val_loader=valid_data_loader, epochs=1, device=device, logfile=logfile)

In [34]:
test_data_loader = DataLoader(torch.utils.data.TensorDataset(torch.from_numpy(df_test2["Book_id"].values), torch.from_numpy(df_test2["NumericID"].values), torch.from_numpy(df_test2["NumericRating"].values)), batch_size=1028, shuffle=False, drop_last=False)
model.eval()
target_list = []
pred_list = []
with torch.no_grad():
    for batch_idx, batch in enumerate(test_data_loader):
        books,users, ratings = batch
        books = books.to(device)
        users = users.to(device)
        out = model(books, users)
        pred_list.append(out.to('cpu').detach().numpy())
        target_list.append(ratings.to('cpu').detach().numpy())
    targets = np.concatenate(target_list)
    preds = np.concatenate(pred_list)
    valid_acc = accuracy2(preds, targets)
print(valid_acc)

0.6773505771297724


This performed pretty well. I would like to see if 2 epochs of training would yield better results

I removed the output for the following cell to aid with the readability.

In [ ]:
model = BookRecommender(books_size=books_size, users_size=users_size, pdropbooks=0.7, pdropusers = 0.1, pdropfc=0.5, books_dim=12,  users_dim=12, fchidden1=16, fchidden2=32, lr =0.00217058)
books_size=5860902
users_size=30725236
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")
model.load_state_dict(torch.load('models/best_ret_checkpoint_fullytrained.pt'))
train_data_loader = DataLoader(torch.utils.data.TensorDataset(torch.from_numpy(df_train3["Book_id"].values), torch.from_numpy(df_train3["NumericID"].values), torch.from_numpy(df_train3["NumericRating"].values)), batch_size=128, shuffle=True, drop_last=True)
valid_data_loader = DataLoader(torch.utils.data.TensorDataset(torch.from_numpy(df_valid2["Book_id"].values), torch.from_numpy(df_valid2["NumericID"].values), torch.from_numpy(df_valid2["NumericRating"].values)), batch_size=128, shuffle=False, drop_last=False)
model.compile(device=device)
logfile = open('models/Amazonhistory3.log', 'at', buffering=1)
hist = model.train_model(train_data_loader,val_loader=valid_data_loader, epochs=2, device=device, logfile=logfile)

In [36]:
test_data_loader = DataLoader(torch.utils.data.TensorDataset(torch.from_numpy(df_test2["Book_id"].values), torch.from_numpy(df_test2["NumericID"].values), torch.from_numpy(df_test2["NumericRating"].values)), batch_size=1028, shuffle=False, drop_last=False)
model.eval()
target_list = []
pred_list = []
with torch.no_grad():
    for batch_idx, batch in enumerate(test_data_loader):
        books,users, ratings = batch
        books = books.to(device)
        users = users.to(device)
        out = model(books, users)
        pred_list.append(out.to('cpu').detach().numpy())
        target_list.append(ratings.to('cpu').detach().numpy())
    targets = np.concatenate(target_list)
    preds = np.concatenate(pred_list)
    valid_acc = accuracy2(preds, targets)
print(valid_acc)

0.7062537969063974


After retraining the model for two epochs, the DNN performs surprisingly well. I was able to recommend a book with 70.62% accuracy! This model could be used to create a recommendation system for independant bookstores or anyone with an interest in books. The model has a very large database and can easily be adapted to use with new users.